# Chapter 8: Calculation of Molecular Properties

## 8.6. Time-Dependent Calculation

In the following section, we will perform time-dependent calculation to predict the absorption spectrum of coumarin:

### 8.6.1. Time-Dependent Calculation

In [ ]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from utils import View3DModel
import psi4
import py3Dmol

In [ ]:
# Define the molecule
smiles = 'O=C1OC2=C(C=CC=C2)C=C1'
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, useRandomCoords=True)
AllChem.UFFOptimizeMolecule(mol, maxIters=200)

In [ ]:
# View 3D model with py3Dmol
View3DModel(mol)

In [ ]:
# Set the number of threads and memory limit
psi4.set_num_threads(16)
psi4.set_memory(16*1024*1024*1024) # 16 GB

In [ ]:
# Set up the basis set, temperature and pressure
psi4.set_options({'BASIS': 'cc-pvdz',
                  'TDSCF_STATES': 10,
                  'ROOTS_PER_IRREP': [5]})

In [ ]:
# Write the geometry to XYZ string
xyz_string = Chem.MolToXYZBlock(mol)

# Get the psi4 geometry
geometry = psi4.geometry(xyz_string)

# Perform geometry optimization
psi4.optimize('b3lyp', molecule=geometry, return_wfn=True)

In [ ]:
# Run the TD calculation
energy, wfn = psi4.energy('td-scf', return_wfn=True, molecule=geometry)

### 8.6.2. Prediction of Absorption Spectra

After time-dependent calculation, the excitation energies and oscillator strengths can be extracted for predicting absorption spectrum.

In [ ]:
# Extract information about the excited states
states_info = psi4.procrouting.response.scf_response.tdscf_excitations(wfn, states=5)

In [ ]:
# Iterate through the states and gather the wavelengths and oscillator strengths
exc_energies = []
osc_strengths = []
for state in states_info:
    exc_energies.append(state['EXCITATION ENERGY'])
    osc_strengths.append(state['OSCILLATOR STRENGTH (LEN)'])

In [ ]:
# Convert excitation energies (Hartrees) to wavelengths (nm) using E = hc/λ
wavelengths_nm = [(psi4.constants.h * psi4.constants.c * 1e9) / (energy * psi4.constants.hartree2ev * psi4.constants.electron_volt) for energy in exc_energies]

# Function to generate a Gaussian curve
def gaussian(x, mu, sigma, intensity):
    return intensity * np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))

# Generate a continuous spectrum
spectrum_x = np.linspace(100, 800, 1000)  # Adjust range and resolution as needed
spectrum_y = np.zeros_like(spectrum_x)

# Width of the Gaussian peaks (standard deviation)
gaussian_width = 5  # You can adjust this value

# Superimpose Gaussian curves for each transition
for wavelength, strength in zip(wavelengths_nm, osc_strengths):
    spectrum_y += gaussian(spectrum_x, wavelength, gaussian_width, strength)

# Create the spectrum plot
plt.figure(figsize=(10, 6))
plt.plot(spectrum_x, spectrum_y, color='blue')
plt.title('Predicted Absorption Spectrum of Coumarin')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Relative Intensity')
plt.xlim(100, 800)
plt.ylim(0, max(spectrum_y) * 1.1)
plt.show()